In [ ]:
import re
import json
import csv
import tweepy
import pytz
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from csv import writer

In [ ]:
consumer_key=environ['CONSUMER_KEY']
consumer_secret=environ['CONSUMER_SECRET']
access_token=environ['ACCESS_TOKEN']
access_token_secret=environ['ACCESS_TOKEN_SECRET']

In [ ]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# set access to user's access key and access secret 
auth.set_access_token(access_token, access_token_secret)

In [ ]:
api = tweepy.API(auth)

In [ ]:
date_obj = datetime.now()

In [ ]:
tzconfig = pytz.timezone("America/Fortaleza")

In [7]:
date_from = datetime(2022,1,16)
date_from = date_from.astimezone(tzconfig)
date_to = datetime(2023,2,1)
date_to = date_to.astimezone(tzconfig)

In [ ]:
date_from = date_obj - timedelta(days=1)
date_from_str = date_from.strftime('%Y-%m-%d')
date_from = datetime.strptime(date_from_str, '%Y-%m-%d')
date_from = date_from.astimezone(tzconfig)

In [ ]:
date_to_str = date_obj.strftime('%Y-%m-%d')
date_to = datetime.strptime(date_to_str, '%Y-%m-%d')
date_to = date_to.astimezone(tzconfig)

In [10]:
#date_to = date_obj - timedelta(days=1)
#date_to_str = date_to.strftime('%Y-%m-%d')
#date_to = datetime.strptime(date_to_str, '%Y-%m-%d')
#date_to = date_to.astimezone(tzconfig)

In [ ]:
tweets = []

In [12]:
#agg_tweets = []

## Postgres connection

In [ ]:
def connection():
    
    conn = None
    
    try:
        conn = psycopg2.connect(
        host="localhost",
        database="report_db",
        user="postgres",
        password="postgres")
    except Exception as error:
        print(error)
    finally:
        if conn:
            return conn

## Insert tweets

In [ ]:
def insert_tweets(data):
    
    conn = None
    rows = None
    
    sql = """
        INSERT INTO tweets(date,tweet_text,candidate_id)
        Values(%s,%s,%s)
    """
    
    try:
        conn = connection()
        cur = conn.cursor()
        cur.executemany(sql, data)
        rows = cur.rowcount
        conn.commit()
        cur.close()
    except Exception as error:
        print(error)
    finally:
        if conn:
            conn.close()

## Insert into agg_tweets

In [ ]:
def insert_agg(data):
    
    conn = None
    rows = None
    
    sql = """
        INSERT INTO aggregate_tweets(date,followers_count,tweets_count,word_count,hashtags,retweets,likes,candidate_id)
        Values(%s,%s,%s,%s,%s,%s,%s,%s)
    """
    
    try:
        conn = connection()
        cur = conn.cursor()
        cur.executemany(sql, data)
        rows = cur.rowcount
        conn.commit()
        cur.close()
    except Exception as error:
        print(error)
    finally:
        if conn:
            conn.close()
            if rows:
                print(f'Rows inserted: {rows}')

### Write csv

## Word count

In [ ]:
def write_raw_tweets(data):
    with open('raw_tweets.csv', 'a', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        for d in data:
            writer.writerow(d) 

In [ ]:
def write_agg_tweets(data):
    with open('aggregate_tweets.csv', 'a', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        for d in data:
            writer.writerow(d) 

In [ ]:
def word_count(tweet_text):
    # Cleaning
    tweet_text = re.sub("@[A-Za-z0-9_]+","", tweet_text)
    tweet_text = re.sub("#[A-Za-z0-9_]+","", tweet_text)
    tweet_text = re.sub(r"http\S+", "", tweet_text)
    tweet_text = re.sub(r"www.\S+", "", tweet_text)
    tweet_text = re.sub('[()!?]', ' ', tweet_text)
    tweet_text = re.sub('\[.*?\]',' ', tweet_text)
    tweet_text = re.sub(r'[^\w\s]', '', tweet_text)
    tweet_text = tweet_text.split()
    
    return len(tweet_text)

## Tweets text

In [ ]:
def get_tweets(candidate_id, tweets, api, date_from, date_to):
    hashtags = ''
    tweets_raw = []
    
    for t in tweets:
            date = t.created_at
            date = date.astimezone(tzconfig)
            if date > date_from and date < date_to:
                status = api.get_status(t.id, tweet_mode = "extended")
                text = status.full_text 
                word = word_count(text)
                retweet_count = t.retweet_count
                favorite_count = t.favorite_count
                if t.entities['hashtags']:
                    hash_list = t.entities['hashtags']
                    for h in hash_list:
                        if hashtags:
                            hashtags += ','+h['text']
                        else:
                            hashtags = h['text']
                            
                tweets_raw.append([date.strftime('%Y-%m-%d %H:%M:%S'), text, word, hashtags, retweet_count, favorite_count, candidate_id])
                
                
    return tweets_raw

## Hashtags

In [ ]:
def get_hashtags(hashtags):
    words = ''
    hashtags = set(hashtags)
    
    for h in hashtags:
        if words:
            words += ', '+h
        else:
            words = h
    
    return words

## Tweets metrics

In [ ]:
def tweets_metrics(date_from_str, followers_count, tweets, api, date_from, date_to, candidate_id):
    tweets_count = 0
    tweets_text = ''
    retweet_count = 0 
    favorite_count = 0
    hashtags = []
    
    for t in tweets:
        date = t.created_at
        date = date.astimezone(tzconfig)
        if date > date_from and date < date_to:
            status = api.get_status(t.id, tweet_mode = "extended")
            text = status.full_text 
            tweets_count += 1 
            tweets_text += text.lower()
            retweet_count += t.retweet_count
            favorite_count += t.favorite_count
            if t.entities['hashtags']:
                hash_list = t.entities['hashtags']
                for h in hash_list:
                    hashtags.append(h['text'])
    if hashtags:
        hashtags = get_hashtags(hashtags)
    else:
        hashtags = ''
                        
    tweet_word_count = word_count(tweets_text)                     
        
    return [date_from_str, followers_count, tweets_count, tweet_word_count, hashtags, retweet_count, favorite_count, candidate_id]

# Bolsonaro

### Followers count

In [ ]:
bolsonaro = api.get_user(screen_name='jairbolsonaro')
bolsonaro_followers = bolsonaro.followers_count
bolsonaro_followers

### Raw tweets

In [ ]:
bolsonaro_tweets = api.user_timeline(screen_name='jairbolsonaro', count=5000, include_rts=False, exclude_replies=True)

In [ ]:
raw_tweets.extend(get_tweets(22, bolsonaro_tweets, api, date_from, date_to))  

In [ ]:
get_tweets(22, bolsonaro_tweets, api, date_from, date_to)

### Metrics

In [ ]:
bolsonaro_metrics = tweets_metrics(date_from_str, bolsonaro_followers, bolsonaro_tweets, api, date_from, date_to, 22)
agg_tweets.append(bolsonaro_metrics)

# Lula

### Followers count

In [ ]:
lula = api.get_user(screen_name='LulaOficial')
lula_followers = lula.followers_count
lula_followers

### Raw tweets

In [ ]:
lula_tweets = api.user_timeline(screen_name='LulaOficial', count=5000, include_rts=False, exclude_replies=True)

In [ ]:
raw_tweets.extend(get_tweets(13, lula_tweets, api, date_from, date_to))  

### Metrics

In [ ]:
lula_metrics = tweets_metrics(date_from_str, lula_followers, lula_tweets, api, date_from, date_to, 13)
agg_tweets.append(lula_metrics)

## Insert

In [ ]:
insert_raw(raw_tweets)